# Node.js 异步编程

Node.js 中的 API 有同步 API 和异步 API 两种：

- 同步 API：只有当前 API 执行完成后，才能继续执行下一个 API
- 异步 API：当前 API 的执行不会阻塞后续代码的执行

如下：

In [1]:
// 同步 API
console.log('before');
console.log('after');

// 异步 API
console.log('before');
setTimeout(() => {
    console.log('last');
}, 2000);
console.log('after');

before
after
before
after
last


Node.js 会从上到下依次执行代码，遇到同步 API 把它拿到**同步代码执行区**中去执行，遇到异步 API 它不会去执行，而是把它放到**异步代码执行区**中，当代码中所有的同步代码都执行完之后，再到异步代码执行区中去依次执行代码，当异步代码执行完之后，系统会去**回调函数队列**中去找这个异步 API 所对应的回调函数，然后把回调函数放到同步代码执行区中再去执行它。

同步 API 可以从返回值中拿到 API 执行的结果，但是异步 API 是不可以的，如下：

In [2]:
// 同步
function sum (n1, n2) {
    return n1 + n2;
}
const result = sum(10, 20);
console.log(result);

// 异步
function getMsg () {
    setTimeout(() => {
        return {msg: 'Hello Node.js'};
    }, 2000);
}
const msg = getMsg();
console.log(msg);

30
undefined


异步 API 可以通过回调函数拿到 API 执行结果。

## 回调函数

回调函数即自己定义函数让别人去调用。如下：

In [3]:
function getData (callback) {
    callback('123');
}

getData((n) => {
    console.log('callback函数被调用了');
    console.log(n);
})

callback函数被调用了
123


让我们使用回调函数来接收上一段代码的执行结果，如下：

In [4]:
function getMsg (callback) {
    setTimeout(() => {
        callback({msg: 'Hello Node.js'});
    }, 2000);
}
getMsg((data) => {
    console.log(data);
});

{ msg: 'Hello Node.js' }


如果异步 API 后面代码的执行依赖当前异步 API 的执行结果，但实际上后续代码在执行的时候异步 API 还没有返回结果，这个问题要怎么解决呢？

In [5]:
const fs1 = require('fs');

fs.readFile('../README.md', 'utf8', (err, result) => {});
console.log('文件读取结果');

文件读取结果


我们希望文件读取完成之后打印文件读取结果，一个显然的方法是把后续代码放到异步 API 的回调函数里，但如果要**依次**读取 A 文件、B 文件、C 文件，这种方法就会导致回调层次嵌套太多，这被称为**回调地狱（callback hell）**。如下：

In [6]:
const fs2 = require('fs');

fs2.readFile('js.ipynb', 'utf8', (err, result1) => {
  console.log(result1.slice(212, 225));
  fs2.readFile('es5.ipynb', 'utf8', (err, result2) => {
    console.log(result2.slice(87, 93));
    fs2.readFile('es6.ipynb', 'utf8', (err, result3) => {
      console.log(result3.slice(87, 93));
    })
  })
})

JavaScript 基础
ES5 语法
ES6 语法


## promise

ES6 中提供的 `promise` 就是为了解决回调地狱问题的，`promise` 实际上是一个构造函数，如下：

In [7]:
let promise = new Promise((resolve, reject) => {
  setTimeout(() => {
    if(true){
      resolve({name: '张三'});
    }else{
      reject('失败了');
    }
  }, 2000);
})

promise.then(result => console.log(result)).catch(err => console.log(err))

{ name: '张三' }


让我们使用 promise 来改写上面的依次读取文件的例子，如下：

In [8]:
const fs3 = require('fs');

function p1(){
  return new Promise((resolve, reject) => {
    fs3.readFile('js.ipynb', 'utf8', (err, result) => {
      if(err != null){
        reject(err)
      }else{
        resolve(result.slice(212, 225))
      }
    })
  })
}

function p2(){
  return new Promise((resolve, reject) => {
    fs3.readFile('es5.ipynb', 'utf8', (err, result) => {
      if(err != null){
        reject(err)
      }else{
        resolve(result.slice(87, 93))
      }
    })
  })
}

function p3(){
  return new Promise((resolve, reject) => {
    fs3.readFile('es6.ipynb', 'utf8', (err, result) => {
      if(err != null){
        reject(err)
      }else{
        resolve(result.slice(87, 93))
      }
    })
  })
}

p1().then(result => {
  console.log(result);
  return p2();
}).then(result => {
  console.log(result);
  return p3();
}).then(result => {
  console.log(result);
})

JavaScript 基础
ES5 语法
ES6 语法


## async await

ES7 引入

让我们使用 async 和 await 来改写上面的依次读取文件的例子，如下：

In [9]:
const fs4 = require('fs');
const promisify = require('util').promisify;
const readFile = promisify(fs4.readFile);

async function run(){
  let r1 = await readFile('js.ipynb', 'utf8');
  let r2 = await readFile('es5.ipynb', 'utf8');
  let r3 = await readFile('es6.ipynb', 'utf8');
  console.log(r1.slice(212, 225));
  console.log(r2.slice(87, 93));
  console.log(r3.slice(87, 93));
}

run();

JavaScript 基础
ES5 语法
ES6 语法
